In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
import time
from tensorflow.keras.layers import Dense
warnings.filterwarnings("ignore")

# Simulation data

In [35]:
data = pd.read_csv("/kaggle/input/simulationdata/Simulationdata.csv",index_col= 0)
data

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,feature91_k,feature92_k,feature93_k,feature94_k,feature95_k,feature96_k,feature97_k,feature98_k,feature99_k,y
0,0.461406,-0.138062,0.678772,1.527778,-0.218823,-0.145966,1.500183,0.711481,-0.433734,0.501047,...,-0.979433,0.463950,-0.384183,0.512113,0.015657,-0.690012,-0.558037,-0.565508,2.721553,1
1,-1.059556,0.807638,-1.180781,0.244734,-2.025435,-1.232025,0.165041,0.682626,0.208686,-0.157825,...,-1.279174,0.652524,-0.121155,-0.345293,1.321991,0.690405,0.350790,0.369794,0.931040,0
2,-0.095075,0.985780,0.394094,-0.589024,0.404716,1.613266,-0.059700,1.505561,-2.589300,0.780671,...,0.694236,0.497791,0.024633,0.594199,-0.417800,-1.647849,0.540547,0.611145,-0.013600,1
3,-0.254152,-1.395087,-0.384428,-0.293781,-0.813647,-0.073646,0.365155,1.825841,0.211904,0.215750,...,-2.084411,1.355300,-0.607549,-0.210499,1.020312,-0.815541,-1.246425,-1.357394,0.573031,0
4,-1.064050,0.464173,-0.880810,1.554045,-0.793729,-0.233248,0.760637,-1.278976,0.264916,1.266400,...,1.040937,-1.547896,1.017070,-0.096840,2.970988,-1.034747,0.388984,1.671235,-0.286414,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-2.719838,0.245937,-0.380494,-0.174807,-0.923690,-0.060646,1.319356,-0.497589,1.004847,-0.849204,...,-1.261415,1.671708,1.207839,3.024718,3.153122,0.406847,-0.757302,-1.000496,0.115265,0
496,-1.249367,0.019293,0.859484,0.192001,0.329237,1.219954,-0.738992,-0.272049,1.675527,-0.239070,...,1.068995,-1.610087,1.177215,-1.067030,-0.069279,-0.856657,0.215517,-0.481231,1.749924,0
497,0.559587,0.877735,1.646569,1.005932,0.093156,-1.212238,0.125245,0.202628,0.747672,1.313196,...,0.004768,0.109657,-0.461855,1.119496,0.730847,-1.192008,1.568252,-0.299782,-0.661093,1
498,-1.515261,0.175513,0.699545,-1.373012,-0.187130,0.797151,-2.858676,1.277270,0.391446,0.675390,...,0.115898,0.706394,0.489416,-1.144433,1.015803,0.695032,0.517188,-1.165706,2.305960,0


In [4]:
import torch
import torch.autograd as autograd


def add_dimension_glasso(var, dim=0):
    return var.pow(2).sum(dim=dim).add(1e-8).pow(1/2.).sum()


def deep_lasso_regularizer(loss, inputs):
    grad_params = autograd.grad(loss, inputs, create_graph=True, allow_unused=True)
    regval = add_dimension_glasso(grad_params[0], dim=0)
    return regval


def get_feat_importance_deeplasso(net, testloader, criterion, device):
    net.eval()
    grads = []
    for batch_idx, (inputs_num, inputs_cat, targets) in enumerate(testloader):
        inputs_num, inputs_cat, targets = inputs_num.to(device).float(), inputs_cat.to(device), targets.to(device)
        inputs_num, inputs_cat = inputs_num if inputs_num.nelement() != 0 else None, \
                                    inputs_cat if inputs_cat.nelement() != 0 else None
        inputs_num.requires_grad_()
        outputs = net(inputs_num, inputs_cat)
        loss = criterion(outputs, targets)
        grad_params = autograd.grad(loss, inputs_num, create_graph=True, allow_unused=True)
        grads.append(grad_params[0].detach().cpu())

    grads = torch.cat(grads)
    importances = grads.pow(2).sum(dim=0).pow(1/2.)
    return importances

In [36]:
data_re = data.iloc[:,:100]
data_re

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var90,var91,var92,var93,var94,var95,var96,var97,var98,var99
0,0.461406,-0.138062,0.678772,1.527778,-0.218823,-0.145966,1.500183,0.711481,-0.433734,0.501047,...,-1.139589,-0.118937,0.461903,0.033938,1.317960,0.851600,-0.061595,0.272244,0.350472,0.385494
1,-1.059556,0.807638,-1.180781,0.244734,-2.025435,-1.232025,0.165041,0.682626,0.208686,-0.157825,...,-0.366453,0.927959,0.801559,1.383283,-1.018771,-0.336806,0.735571,0.514277,1.590418,0.402996
2,-0.095075,0.985780,0.394094,-0.589024,0.404716,1.613266,-0.059700,1.505561,-2.589300,0.780671,...,-0.972883,0.181779,0.408541,-1.101319,0.016053,0.699702,-1.251158,1.201414,-0.324483,1.252936
3,-0.254152,-1.395087,-0.384428,-0.293781,-0.813647,-0.073646,0.365155,1.825841,0.211904,0.215750,...,1.032993,-1.396659,0.603730,-1.675585,-0.866563,-1.261675,0.759452,-0.022832,0.040496,-1.147744
4,-1.064050,0.464173,-0.880810,1.554045,-0.793729,-0.233248,0.760637,-1.278976,0.264916,1.266400,...,0.728636,0.442393,-1.713243,0.999105,0.908532,0.911316,0.333963,0.275905,1.089121,1.929070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-2.719838,0.245937,-0.380494,-0.174807,-0.923690,-0.060646,1.319356,-0.497589,1.004847,-0.849204,...,0.409179,-1.047576,2.274519,1.565375,2.044504,2.358369,-0.713754,-0.879213,2.085462,-0.167242
496,-1.249367,0.019293,0.859484,0.192001,0.329237,1.219954,-0.738992,-0.272049,1.675527,-0.239070,...,-0.315627,-1.131946,-0.243462,0.005096,1.240280,-1.078799,0.317837,0.626911,0.032810,-0.529699
497,0.559587,0.877735,1.646569,1.005932,0.093156,-1.212238,0.125245,0.202628,0.747672,1.313196,...,-0.453492,0.024436,-1.320878,1.907564,0.905234,-0.718263,-1.216225,-1.042418,0.642308,-1.049911
498,-1.515261,0.175513,0.699545,-1.373012,-0.187130,0.797151,-2.858676,1.277270,0.391446,0.675390,...,-0.763045,-1.129448,0.636055,0.906020,1.198538,-0.257622,0.434496,0.940782,-1.300555,0.258037


In [6]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
X = data_re.values
y = data['y'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)  

# 创建DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, _=None):  
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# defind model
input_dim = X.shape[1]
hidden_dim = 264
output_dim = len(set(y))  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleMLP(input_dim, hidden_dim, output_dim).to(device)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

model.train()
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


In [9]:
def add_dimension_glasso(var, dim=0):
    return var.pow(2).sum(dim=dim).add(1e-8).pow(1/2.).sum()

def deep_lasso_regularizer(loss, inputs):
    grad_params = autograd.grad(loss, inputs, create_graph=True, allow_unused=True)
    regval = add_dimension_glasso(grad_params[0], dim=0)
    return regval

def get_feat_importance_deeplasso(net, testloader, criterion, device):
    net.eval()
    grads = []
    for batch_idx, (inputs_num, targets) in enumerate(testloader):
        inputs_num, targets = inputs_num.to(device), targets.to(device)
        inputs_num.requires_grad_()
        outputs = net(inputs_num)
        loss = criterion(outputs, targets)
        reg_loss = deep_lasso_regularizer(loss, inputs_num)  # 使用 deep_lasso_regularizer 函数
        grad_params = autograd.grad(reg_loss, inputs_num, create_graph=True, allow_unused=True)
        grads.append(grad_params[0].detach().cpu())

    grads = torch.cat(grads)
    importances = grads.pow(2).sum(dim=0).pow(1/2.)
    return importances

test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

# calculate features importance
importances = get_feat_importance_deeplasso(model, test_loader, criterion, device)

feature_names = data_re.columns
importance_indices = torch.argsort(importances, descending=True)
top_33_indices = importance_indices[:33]
top_33_features = feature_names[top_33_indices]

print("Top 33 Important Features:")
for i, feature in enumerate(top_33_features):
    print(f"{i+1}: {feature} - Importance: {importances[top_33_indices[i]].item()}")


Top 33 Important Features:
1: var27 - Importance: 0.05092018470168114
2: var23 - Importance: 0.04833841323852539
3: var10 - Importance: 0.04515768587589264
4: var26 - Importance: 0.04285818338394165
5: var6 - Importance: 0.04181760549545288
6: var16 - Importance: 0.03413280099630356
7: var31 - Importance: 0.0333033986389637
8: var24 - Importance: 0.0319250151515007
9: var1 - Importance: 0.03162641450762749
10: var30 - Importance: 0.028380122035741806
11: var25 - Importance: 0.02618580311536789
12: var7 - Importance: 0.02521054819226265
13: var22 - Importance: 0.022943640127778053
14: var13 - Importance: 0.021216843277215958
15: var71 - Importance: 0.018839480355381966
16: var20 - Importance: 0.017960915341973305
17: var11 - Importance: 0.017819978296756744
18: var19 - Importance: 0.017170941457152367
19: var32 - Importance: 0.016565166413784027
20: var0 - Importance: 0.014993754215538502
21: var29 - Importance: 0.014234738424420357
22: var50 - Importance: 0.013746044598519802
23: var28

## Prediction

In [37]:
def normal_prediction(X_train,y_train,X_test,y_test,input_s):
    input_dim = input_s
    running_times = []
    acc= []
    for i in range(0,10,1):
        model = keras.Sequential()
        model.add(layers.Dense(10, input_shape=(input_dim,)))

        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        start_time = time.time()
        model.fit(X_train,y_train, epochs=250, batch_size=32, verbose=0)
        predictions = model.predict(X_test)
        predicted = (predictions > 0.5)  
        accuracy = accuracy_score(y_test, predicted) 
       
        end_time = time.time()
        run_time = end_time - start_time
        running_times.append(run_time)
        acc.append(accuracy)
    avg_running_time = np.mean(running_times)
    avg_accuracy = np.mean(acc)

    print("Average running time:", avg_running_time, "s")
    print("Average accuracy:", avg_accuracy * 100, "%")

In [40]:
from sklearn.model_selection import train_test_split
var_t = data.loc[:, ['var27',
'var23',
'var10',
'var26',
'var6',
'var16',
'var31',
'var24',
'var1',
'var30',
'var25',
'var7',
'var22',
'var13',
'var71',
'var20',
'var11',
'var19',
'var32',
'var0','y']]

X_train, X_test, y_train, y_test = train_test_split(var_t.drop(columns=['y']), var_t['y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)
    
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Average running time: 11.738960337638854 s
Average accuracy: 82.0 %


## Application data

In [20]:
from sklearn.preprocessing import MinMaxScaler
df = pd.read_excel('/kaggle/input/applicationdata/cancer.xlsx')
data = df.rename(columns={'Column1': 'ID', 'Column2': 'Y', 'Column3': 'radius1', 'Column4': 'texture1', 'Column5': 'perimeter1', 
                     'Column6': 'area1', 'Column7': 'smoothness1', 'Column8': 'compactness1', 'Column9': 'concavity1', 'Column10': 'concave_points1',
                     'Column11': 'symmetry1', 'Column12': 'fractal_dimension1', 'Column13': 'radius2', 'Column14': 'texture2', 'Column15': 'perimeter2', 
                     'Column16': 'area2', 'Column17': 'smoothness2', 'Column18': 'compactness2', 'Column19': 'concavity2', 'Column20': 'concave_points2',
                     'Column21': 'symmetry2', 'Column22': 'fractal_dimension2', 'Column23': 'radius3', 'Column24': 'texture3', 'Column25': 'perimeter3', 
                     'Column26': 'area3', 'Column27':'smoothness3', 'Column28': 'compactness3', 'Column29': 'concavity3', 'Column30': 'concave_points3',
                     'Column31': 'symmetry3', 'Column32': 'fractal_dimension3'})
data['Y'] = data['Y'].replace({'B': 0, 'M': 1})
# features_to_normalize = ['var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var0']
cols_to_normalize = data.columns[2:]

# Normalize the selected columns using Min-Max scaling
scaler = MinMaxScaler()
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])
data

,ID,Y,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,1,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,842517,1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,84300903,1,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,84348301,1,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,84358402,1,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,1,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,926682,1,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,926954,1,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,927241,1,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


In [21]:
X = data.drop(columns=['Y']).values
y = data['Y'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)  

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [22]:
input_dim = X.shape[1]
hidden_dim = 10
output_dim = len(set(y))

model = SimpleMLP(input_dim, hidden_dim, output_dim).to(device)

In [23]:
num_epochs = 50

model.train()
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


In [24]:
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

importances = get_feat_importance_deeplasso(model, test_loader, criterion, device)

feature_names = data.drop(columns=['Y']).columns

importance_indices = torch.argsort(importances, descending=True)

top_33_indices = importance_indices[:33]
top_33_features = feature_names[top_33_indices]

print("Top 33 Important Features:")
for i, feature in enumerate(top_33_features):
    print(f"{i+1}: {feature} - Importance: {importances[top_33_indices[i]].item()}")

Top 33 Important Features:
1: smoothness2 - Importance: 0.00012321244867052883
2: concavity2 - Importance: 0.00011928225285373628
3: fractal_dimension3 - Importance: 0.00010587593715172261
4: radius1 - Importance: 0.0001035825043800287
5: concavity1 - Importance: 0.00010283744632033631
6: area1 - Importance: 0.00010024413495557383
7: texture1 - Importance: 9.941057214746252e-05
8: symmetry3 - Importance: 9.011509246192873e-05
9: fractal_dimension1 - Importance: 8.62941742525436e-05
10: smoothness1 - Importance: 7.579434895887971e-05
11: concavity3 - Importance: 6.92301255185157e-05
12: concave_points1 - Importance: 4.51921732746996e-05
13: compactness3 - Importance: 4.245576565153897e-05
14: compactness1 - Importance: 4.208533209748566e-05
15: radius3 - Importance: 4.076876939507201e-05
16: area3 - Importance: 3.6814417399000376e-05
17: concave_points3 - Importance: 3.524822386680171e-05
18: symmetry2 - Importance: 3.2606661989120767e-05
19: radius2 - Importance: 2.9302635084604844e-05

## Prediction

In [25]:
def normal_prediction(X_train,y_train,X_test,y_test,input_s):
    input_dim = input_s
    running_times = []
    acc= []
    for i in range(0,10,1):
        model = keras.Sequential()
        model.add(layers.Dense(10, input_shape=(input_dim,)))

        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        start_time = time.time()
        model.fit(X_train,y_train, epochs=250, batch_size=32, verbose=0)
        predictions = model.predict(X_test)
        predicted = (predictions > 0.5)  
        accuracy = accuracy_score(y_test, predicted) 
       
        end_time = time.time()
        run_time = end_time - start_time
        running_times.append(run_time)
        acc.append(accuracy)
    avg_running_time = np.mean(running_times)
    avg_accuracy = np.mean(acc)

    print("Average running time:", avg_running_time, "s")
    print("Average accuracy:", avg_accuracy * 100, "%")

In [29]:
var_t = data.loc[:, ['Y',
'smoothness2', 
'concavity2',
'fractal_dimension3',
'radius1',
'concavity1',
'area1',
'texture1',
'symmetry3',
'fractal_dimension1',
'smoothness1',
'concavity3',
'concave_points1']]

X_train, X_test, y_train, y_test = train_test_split(var_t.drop(columns=['Y']), var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [34]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Average running time: 12.646170353889465 s
Average accuracy: 95.6140350877193 %
